In [6]:
!pip install ipykernel
!pip install panda 
!pip install numpy
!pip install scipy
!pip install sklearn
!pip install matplotlib
!pip install nltk

In [13]:
import codecs
import re 
import copy
import collections

import numpy as np
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from __future__ import division
import matplotlib

%matplotlib inline

In [33]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kamel_000\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\kamel_000\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\kamel_000\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     C:\Users\kamel_000\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\kamel_000\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\brown.zip.
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     C:\Users\kamel_000\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     C:\Users\kamel_000\AppData\Roa

KeyboardInterrupt: 

In [34]:
from nltk.corpus import stopwords

In [76]:
#with codecs.open("DONNES/train_cap2018.csv","r",encoding="utf-8") as f:
 #   train = f.read()
df = pd.read_csv("DONNES/train_cap2018.csv")
saved_column = df.fulltext

print(type(saved_column))

<class 'pandas.core.series.Series'>


In [71]:
word_pattern = re.compile("^/w+$")

In [78]:
#Tokenize les articles (you, and, or ...) et les comptes
def get_text_counter(text):
    tokens = WordPunctTokenizer().tokenize(PorterStemmer().stem(text))
    tokens = list(map(lambda x: x.lower(), tokens))
    tokens = [token for token in tokens if re.match(word_pattern,token) and token not in esw]
    return collections.Counter(tokens),len(tokens)


In [79]:
#Fonction qui la fréquence absolue et la fréquence relative des mots les plus utilisés
def make_df(counter, size):
    abs_freq = np.array([el[1] for el in counter])
    rel_freq = abs_freq / size
    index = [el[0] for el in counter]
    df = pd.DataFrame(data=np.array([abs_freq, rel_freq]).T, index=index, columns=["Absolute Frequency", "Relative Frequency"] )
    df.index.name = "Most common words"
    return df


In [82]:
#calcule des mots les plus utilisés dans le fichier train, ça prend du temps donc ici échantillon des 10 mots les plus itlisés
je_counter, je_size = get_text_counter(train)

#make_df(je_counter.most_common(10),je_size)

AttributeError: '_csv.reader' object has no attribute 'lower'